In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [2]:
def memcheck():
    gc.collect()
    memory_info = psutil.virtual_memory()

    # Display the memory information in GB
    total_memory = memory_info.total / (1024 ** 3)
    available_memory = memory_info.available / (1024 ** 3)
    used_memory = memory_info.used / (1024 ** 3)

    print(f"Total Memory: {total_memory:.2f} GB")
    print(f"Available Memory: {available_memory:.2f} GB")
    print(f"Used Memory: {used_memory:.2f} GB")

def time_taken(start_time, end_time):
    time_taken = end_time - start_time
    hours, remainder = divmod(time_taken.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Time Taken: {hours} hours, {minutes} minutes, {seconds} seconds")

In [34]:
def display_text_in_context(text, words, window = 75):
#Finds the first occurrence of each word in target text and displays it
    
    for word in words:
        ind = text.find(word)
        if ind == -1:
            print (f"{word} not found in text")
        else:
            display_text = text[ind-window:ind+window+1]
            display_text = display_text.replace("\n", " ")
            display_text = display_text.replace("\t", " ")
            display_text = display_text.replace("  ", " ")
            print(display_text)

def display_words_in_context(text_words, words, repeat = False, window = 2):
# Finds the first occurrence of each word in target list of words. Optionally finds every subsequent occurrence     
    if not repeat:
        for word in words:
            try:
                #print("trying ", word)
                index = text_words.index(word)
                #print(f"The index of '{word}' is: {index}")
                display_text = ' '.join(text_words[index - window:index])
                display_text = display_text + " [" + word + "] "
                display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                print(display_text)
            except ValueError:
                print(f"'{word}' not found in the text.")
    else:
        for word in words:
            indices = [index for index, value in enumerate(text_words) if value == word]
            if len(indices) == 0:
                print(f"'{word}' not found in the text.")
            else:
                for index in indices:
                    display_text = ' '.join(text_words[index - window:index])
                    display_text = display_text + " [" + word + "] "
                    display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                    print(display_text)
                

In [35]:
def where_is(word):
    # finds if spell checker has flagged a word.  Must run spell checker first.
    if (word in spell_set_0): 
        print ("In spell set 0")
    if (word in blob_set_0):
        print ("In blob set 0")
    if (word in spell_set_0) or (word in blob_set_0):
        display_words_in_context(test_words_0, [word], False, 6)
    if (word in spell_set_1): 
        print ("In spell set 1")
    if (word in blob_set_1):
        print ("In blob set 1")
    if (word in spell_set_1) or (word in blob_set_1):
        display_words_in_context(test_words_1, [word], False, 6)
    if (word in spell_set_2): 
        print ("In spell set 2")
    if (word in blob_set_2):
        print ("In blob  set 2")
    if (word in spell_set_2) or (word in blob_set_2):
        display_words_in_context(test_words_2, [word], False, 6)
    if (word in spell_set_3): 
        print ("In spell set 3")
    if (word in blob_set_3):
        print ("In blob set 3")
    if (word in spell_set_3) or (word in blob_set_3):
        display_words_in_context(test_words_3, [word], False, 6)        
        

In [5]:
def find_hyphenated_words(text):
    # Use regex to find words that are hyphenated across lines
    # Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
    hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

    for first_part, second_part in hyphenated_words:
        print(f"Found broken word: {first_part}-{second_part}")
    print(f"{len(hyphenated_words)} hyphenated words found, text length is: {len(text)}")
    


In [20]:
def replace_hyphenated_words(text, show_changes = False):
    # This regular expression captures words separated by a hyphen, with letters on both sides.
    pattern = r'(\b\w+)-(\w+\b)'

    # Function to handle replacement and printing
    def replacement(match):
        # Original hyphenated word
        original_word = match.group(0)
        # Replacement word (with space instead of hyphen)
        altered_word = match.group(1) + " " + match.group(2)
        
        # Print the hyphenated word that was altered
        if show_changes: print(f"Altered: {original_word} -> {altered_word}")

        return altered_word

    # Replace hyphenated words and call the replacement function
    result = re.sub(pattern, replacement, text)
    
    return result


In [7]:
def replace_hyphenated_words_alt(text, silent = True):
    # Find words that have a hyphen between them
    hyphenated_words = re.findall(r'\b\w+-\w+\b', text)
    counter = 0
    # Replace each hyphenated word with a space between words instead of the hyphen
    for word in hyphenated_words:
        new_word = word.replace('-', ' ')
        text = text.replace(word, new_word)
        counter += 1
        if not silent: print(f"Replaced '{word}' with '{new_word}'")
    print(counter)
    return text

In [8]:
def display_oddities(text):
    # Run this after creating oddities, use the text BEFORE hyphens were removed
    hyphenated_words = re.findall(r'\b\w+-\w+\b', text)
        
    for word in hyphenated_words:
        new_word = word.replace('-', ' ')
        new_word_split = new_word.split(' ')
        #print(word, new_word)
        if new_word_split[0] in oddities or new_word_split[1] in oddities:
            print (word, new_word)
            display_text_in_context(text, [word], window = 40)
            print("\n")

In [9]:
def clean_text(text, make_lower = True):
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    if make_lower:
        text = text.lower()
    # Strip any leading or trailing spaces
    text = text.strip()
    return text

# Example usage
tt = "This  is   an Example  Text with    DOUBLE  SPACES."
cleaned_text = clean_text(tt, False)
print(cleaned_text)


This is an Example Text with DOUBLE SPACES.


In [10]:
def list_to_dict(alternating_list):
    # Use dictionary comprehension with list slicing to group pairs
    return {alternating_list[i]: alternating_list[i+1]+ " " + alternating_list[i+2] for i in range(0, len(alternating_list), 3)}

# Example usage
alternating_list = ['name', 'Alice', 'bum', 'age', '30', 'years', 'city', 'New York', 'duds']
dictionary = list_to_dict(alternating_list)
print(dictionary)


{'name': 'Alice bum', 'age': '30 years', 'city': 'New York duds'}


In [11]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text



In [12]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"
file_path = directory_path+source_texts[1]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day.txt


In [13]:
contractions = "Aren't	Are not Can't	Can not Couldn't	Could not Didn't	Did not Doesn't	Does not Don't	Do not Hadn't	Had not Hasn't	Has not Haven't	Have not Isn't	Is not Mightn't	Might not Mustn't	Must not Needn't	Need not Shan't	Shall not Shouldn't	Should not Wasn't was not Weren't	Were not Won't	Will not Wouldn't	Would not"
contractions = contractions + " I'd	I would You'd	You would He'd	He would She'd	She would It'd	It would We'd	We would They'd	They would Who'd	Who would There'd There would That'd That would" 
contractions = contractions + " I'll	I will You'll	You will He'll	He will She'll	She will It'll	It will We'll	We will They'll	They will Who'll	Who will That'll That will What'll What will"
contractions = contractions + " I've	I have You've	You have He's	He has She's	She has We've	We have They've	They have Who's	Who has Who've Who have"
contractions = contractions + " I'm	I am You're	You are He's	He is She's	She is It's	It is We're	We are They're	They are Who's	Who is There's There is There're There are That's That is"
contractions = clean_text(contractions, True) + " " + clean_text(contractions, False)
contractions = contractions.split(' ')
print(len(contractions))
print(contractions)
contractions = list_to_dict(contractions)
contractions["can't"] = "cannot"
contractions["Can't"] = "Cannot"

348
["aren't", 'are', 'not', "can't", 'can', 'not', "couldn't", 'could', 'not', "didn't", 'did', 'not', "doesn't", 'does', 'not', "don't", 'do', 'not', "hadn't", 'had', 'not', "hasn't", 'has', 'not', "haven't", 'have', 'not', "isn't", 'is', 'not', "mightn't", 'might', 'not', "mustn't", 'must', 'not', "needn't", 'need', 'not', "shan't", 'shall', 'not', "shouldn't", 'should', 'not', "wasn't", 'was', 'not', "weren't", 'were', 'not', "won't", 'will', 'not', "wouldn't", 'would', 'not', "i'd", 'i', 'would', "you'd", 'you', 'would', "he'd", 'he', 'would', "she'd", 'she', 'would', "it'd", 'it', 'would', "we'd", 'we', 'would', "they'd", 'they', 'would', "who'd", 'who', 'would', "there'd", 'there', 'would', "that'd", 'that', 'would', "i'll", 'i', 'will', "you'll", 'you', 'will', "he'll", 'he', 'will', "she'll", 'she', 'will', "it'll", 'it', 'will', "we'll", 'we', 'will', "they'll", 'they', 'will', "who'll", 'who', 'will', "that'll", 'that', 'will', "what'll", 'what', 'will', "i've", 'i', 'have',

In [14]:
for key, value in contractions.items():
    if " " in key or value[0] == " " or value[-1] == " ": print(key, ": ", value)

In [15]:
text_choice = 0


file_path = directory_path+source_texts[text_choice]+" edited_2.txt"
if os.path.isfile(file_path):
    print ("file found at ", file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
else:
    print("file not found at ", file_path)
    file_path = directory_path+source_texts[text_choice]+" edited_1.txt"
    if os.path.isfile(file_path):
        print ("file found at ", file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        print("file not found at ", file_path)
        file_path = directory_path+source_texts[text_choice]+" edited.txt"
        if os.path.isfile(file_path):
            print ("file found at ", file_path)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        else:
            print("file not found at ", file_path)


file not found at  C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - Never Let Me Go edited_2.txt
file found at  C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - Never Let Me Go edited_1.txt


In [16]:
print (len(text))
text_1 = replace_strings_with_context(text, contractions, window=10, silent = True)
print(len(text_1))

514835
made 3516 changes
521967


In [17]:
unhyphenated_alt = replace_hyphenated_words_alt(text_1)

545


In [21]:
test_words_0 = text_1.translate(str.maketrans('', '', string.punctuation)).split()
test_words_1 = text_1.translate(str.maketrans('', '', string.punctuation)).lower().split()
unhyphenated = replace_hyphenated_words(text_1)
test_words_2 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).split()
test_words_3 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [22]:
start_time = datetime.now()
print(f"Starting processing at: {start_time.strftime('%H:%M:%S')}")

# Find invalid words
blob_set_0 = set()
for word in test_words_0:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_0.add(word)

blob_set_1 = set()
for word in test_words_1:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_1.add(word)

blob_set_2 = set()
for word in test_words_2:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_2.add(word)

blob_set_3 = set()
for word in test_words_3:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_3.add(word)
        

end_time = datetime.now()
print(f"Completed processing at: {end_time.strftime('%H:%M:%S')}")
time_taken(start_time, end_time)

print(len(blob_set_0), len(blob_set_1), len(blob_set_2), len(blob_set_3))

Starting processing at: 03:18:12
Completed processing at: 03:40:34
Time Taken: 0 hours, 22 minutes, 21 seconds
1526 1290 1097 860


In [23]:
print(f"errors found in base text: {len(blob_set_0)}")
print(f"errors found with case lowered: {len(blob_set_1)}")
print(f"errors found with hyphens removed:  {len(blob_set_2)}")     
print(f"errors found with hyphens removed and case lowered: {len(blob_set_3)}")     
print(f"errors eliminated by removing hyphens:  {len(blob_set_0 - blob_set_2)} or in lower case: {len(blob_set_1 - blob_set_3)}")
print(f"errors introduced by removing hyphens:  {len(blob_set_2 - blob_set_0)} or in lower case: {len(blob_set_3 - blob_set_1)}")
print(f"errors eliminated by lowering case:  {len(blob_set_0 - blob_set_1)} or without hyphens: {len(blob_set_2 - blob_set_3)}")
print(f"errors introduced by lowering case:  {len(blob_set_1 - blob_set_0)} or without hyphens: {len(blob_set_3 - blob_set_2)}")

errors found in base text: 1526
errors found with case lowered: 1290
errors found with hyphens removed:  1097
errors found with hyphens removed and case lowered: 860
errors eliminated by removing hyphens:  471 or in lower case: 470
errors introduced by removing hyphens:  42 or in lower case: 40
errors eliminated by lowering case:  414 or without hyphens: 365
errors introduced by lowering case:  178 or without hyphens: 128


In [24]:
oddities = sorted(list(blob_set_2-blob_set_0))
print(' '.join(oddities))
#display_words_in_context(test_words_2, oddities, window = 5)


Ex LP Matilda Open abrac assortment bashedup bric castors clones consciousmaybe cumsale dah forksstuff frogging glassed heartedly hike hitch inthe mini misted nighter nighters offs onthe ordinator pom punching scaled seater shapeI sogreat storey tenniswas theshoulder timehad tohearts tracked tête yearold àtête


In [25]:
display_oddities(text_1)

half-heartedly half heartedly
 some kind of joke, because she laughed half-heartedly, then made some quip of he


Ex-changes Ex changes
 "But sometimes, when I think about the Ex-changes now, a lot of it seems a bit o


Matilda-as Matilda as
I chatted with my friend-I think it was Matilda-as cheerfully as possible, and ha


leap-frogging leap frogging
 watching the knight-rather than in the leap-frogging way of draughts. I did not 


pom-pom pom pom
The zip across the top edge had a furry pom-pom to pull it. I would almost sat on


LP-the LP the
gewater. I suppose it was originally an LP-the recording date's 1956-but what I h


scaled-down scaled down
cover picture was what must have been a scaled-down version of the record sleeve.


dah-dah dah dah
 of my voice, and I may even have gone "dah-dah!" as I brought it out and handed 


show-offs show offs
ected as a couple because they were not show-offs. Some others, Sylvia B. and Rog


one-nighter one nighter
with you; other times it was

In [26]:
# Use spellchecker to validate words
# Initialize spell checker
spell = SpellChecker()


# Find words not in the dictionary
spell_set_0 = {word for word in test_words_0 if word not in spell}
spell_set_1 = {word for word in test_words_1 if word not in spell}
spell_set_2 = {word for word in test_words_2 if word not in spell}
spell_set_3 = {word for word in test_words_3 if word not in spell}

print(len(spell_set_0), len(spell_set_1), len(spell_set_2), len(spell_set_3))


724 721 310 308


In [27]:
both_3 = spell_set_3 & blob_set_3

In [28]:
print(len(both_3))

212


In [ ]:

print(' '.join(word for word in both_3))
# display_words_in_context(test_words_2, spell_only_2 - spell_only_1, False, 3)

In [29]:
#copy and extract the ones that could be real errors, paste them back here
errors_raw = "ethe lps thatsnot coupleshinting shapei youknow timehad isi fromme definitelyhad carefulwho thendo neurone thatwe whoeverwas jwho wewere betweenthem coupleand canunzip wasshe toomuch howshe hs itshould gallerys downthey theshoulder keffers tenniswas becausethey kshe inthe iwould theevening consciousmaybe rway whatwas sogreat virginmeaning jeansruth forksstuff kefferss saidhello sostupid alls dyou onthe therewas moulding alost expectthen abouts cwho reallyfit typesomething possiblesthe àtête abrac courseswhich itis southshe doi bric 5s mag itbut cumsale"
errors_raw = errors_raw.split()
errors_raw = sorted(errors_raw)
print(len(errors_raw))
print (errors_raw)

68
['5s', 'abouts', 'abrac', 'alls', 'alost', 'becausethey', 'betweenthem', 'bric', 'canunzip', 'carefulwho', 'consciousmaybe', 'coupleand', 'coupleshinting', 'courseswhich', 'cumsale', 'cwho', 'definitelyhad', 'doi', 'downthey', 'dyou', 'ethe', 'expectthen', 'forksstuff', 'fromme', 'gallerys', 'howshe', 'hs', 'inthe', 'isi', 'itbut', 'itis', 'itshould', 'iwould', 'jeansruth', 'jwho', 'keffers', 'kefferss', 'kshe', 'lps', 'mag', 'moulding', 'neurone', 'onthe', 'possiblesthe', 'reallyfit', 'rway', 'saidhello', 'shapei', 'sogreat', 'sostupid', 'southshe', 'tenniswas', 'thatsnot', 'thatwe', 'theevening', 'thendo', 'therewas', 'theshoulder', 'timehad', 'toomuch', 'typesomething', 'virginmeaning', 'wasshe', 'wewere', 'whatwas', 'whoeverwas', 'youknow', 'àtête']


In [ ]:
counter_spell_1 = 0
counter_spell_2 = 0
counter_spell_3 = 0
counter_blob_1 = 0
counter_blob_2 = 0
counter_blob_3 = 0
counter_both_3 = 0

for word in errors_raw:
    if word in spell_set_1: counter_spell_1 += 1 
    if word in spell_set_2: counter_spell_2 += 1
    if word in spell_set_3: counter_spell_3 += 1
    else: print("not in spell_3: ", word)
    if word in blob_set_1: counter_blob_1 += 1
    if word in blob_set_2: counter_blob_2 += 1
    if word in blob_set_3: counter_blob_3 += 1
    else: print("not in blob_3: ", word)
    if word in both_3: counter_both_3 += 1
    else: print("not in both_3: ", word)
print (counter_spell_1, counter_spell_2, counter_spell_3)
print (counter_blob_1, counter_blob_2, counter_blob_3)
print(counter_both_3)
    

In [30]:
errors_found = (word for word in errors_raw if (" "+ word) in text_1)
errors_found = sorted(errors_found)
print(len(errors_found))
errors_not_found = [word for word in errors_raw if word not in errors_found]
print(len(errors_not_found))

30
38


In [31]:
errors_found_1 = (word for word in errors_not_found if word in test_words_1)
errors_found_1 = sorted(errors_found_1)
print(len(errors_found_1))
errors_not_found_1 = [word for word in errors_not_found if word not in errors_found_1]
print(len(errors_not_found_1))

25
13


In [32]:
errors_found_2 = (word for word in errors_not_found_1 if word in test_words_2)
errors_found_2 = sorted(errors_found_2)
print(len(errors_found_2))
errors_not_found_2 = [word for word in errors_not_found_1 if word not in errors_found_2]
print(len(errors_not_found_2))

11
2


In [33]:
for word in errors_found:
    print(word)
    display_text_in_context(text_1, [word], window = 40)

5s
ons and none of us, not even the Senior 5s, dared make a sound. There was a real
becausethey
ble about us having sex with each other becausethey would then want to have sex w
betweenthem
 The idea the guardians had differences betweenthem, that never occurred to us." 
bric
ut, he put the mags down on top of some bricks stacked outside the boiler hut-som
canunzip
 it is right on the elbow like that, it canunzip. All you have to do is bend your
carefulwho
y and began telling us how we had to be carefulwho we had sex with. Not just beca
coupleshinting
of snogging and touching up, maybe; and coupleshinting they were having proper se
definitelyhad
 that two of the girls I was closest to definitelyhad done it. Laura with Rob D.,
doi
o completing and so that is what he was doing: getting me to describe things to h
fromme
you do not do it that way! Just take it fromme, you do not do it that way! You de
howshe
ago. "None of us stopped to think about howshe felt , Miss Lucy herself. We never


In [ ]:
print(errors_not_found_2)

In [ ]:

suggestions = ['alost', 'coupleand', 'cwho', 'itbut', 'Iwould', 'Jeansruth', 'jwho', 'Keffers', 'Kefferss', 'kshed', 'Lauras', 'LPs', 'Metchley', 'Portway', 'shapei', 'virginmeaning', 'Wainright', 'Walkman', 'Walkmans', 'wavecrest']

In [ ]:
print(len(suggestions))
suggestions_found = (word for word in suggestions if word in text)
suggestions_found = sorted(suggestions_found)
print(len(suggestions_found))
suggestions_not_found = [word for word in suggestions if word not in suggestions_found]
print(len(suggestions_not_found))

In [ ]:
print(suggestions_found)

In [ ]:
errors = ['0f', 'agre', 'al1', 'allhis', 'andthe', 'anysuch', 'arec', 'avery', 'be-that', 'beyondsuch', 'canbe', 'canonly', 'cansee', 'carryon', 'Chainbers', 'confidentia1', 'contemplatin8', 'Dh', "'distinguished',and", 'docrucially', "don'ttake", 'T ell', "Farraday'scircle", 'friendsand', 'gotto', 'het', 'Iarrived', 'Ican', "if'-I", 'ill1terest', "I'mvery", 'Infact', 'istrue', 'it,involved', "it'srather", 'loc ked', 'lookingback', 'lot,you', 'lover,Stevens', 'lto', 'not-seek', 'ofthe', 'Oh,nothing', 'ot', 'owardst', 'putout', 'roas ting', 'seehe', 'self-vident', 'simplyaccepting', 'startseeing', "that'sall", 'thelikes', 'thevery', 'tJ1e', 'tosit', 'verypleased', 'village,sir', 'wer', 'with-a']


In [ ]:
print(len(errors))

In [ ]:
print("distinguishedand" in errors_raw)
print("distinguishedand" in errors)

In [ ]:
errors_tt == errors_raw

In [ ]:
corrections = ['of', 'agree', 'all', 'all his', 'and the', 'any such', 'care', 'a very', 'be that', 'beyond such', 'can be', 'can only', 'can see', 'carry on', 'Chambers', 'confidential', 'contemplating', 'Oh', "'distinguished', and", 'do crucially', "don't take", 'Tell', 'Farradays Circle', 'friends and', 'got to', 'the', 'I arrived', 'I can', 'if I', 'interest', "I'm very", 'In fact', 'is true', 'it involved', "its rather", 'locked', 'looking back', 'lot, you', 'lover, Stevens', 'to', 'not seek', 'of the', 'Oh, nothing', 'to', 'towards', 'put out', 'roasting', 'see he', 'self-evident', 'simply accepting', 'start seeing', "that's all", 'the likes', 'the very', 'the', 'to sit', 'very pleased', 'village, sir', 'were', 'with a']

In [ ]:
set_1 = set(errors_tt)
set_2 = set(errors)
print(len(set_1), len(set_2))
print(set_1 - set_2)
print(set_2 - set_1)

In [ ]:
counter_spell_1 = 0
counter_spell_2 = 0
counter_spell_3 = 0
counter_blob_1 = 0
counter_blob_2 = 0
counter_blob_3 = 0

for word in errors_tt:
    if word in spell_set_1: counter_spell_1 += 1 
    if word in spell_set_2: counter_spell_2 += 1
    if word in spell_set_3: counter_spell_3 += 1
    if word in blob_set_1: counter_blob_1 += 1
    if word in blob_set_2: counter_blob_2 += 1
    if word in blob_set_3: counter_blob_3 += 1
print (counter_spell_1, counter_spell_2, counter_spell_3)
print (counter_blob_1, counter_blob_2, counter_blob_3)
    

In [ ]:
for word in errors_tt:
    if word not in blob_set_3: print(word)

In [ ]:
counter_spell_1 = 0
counter_spell_2 = 0
counter_spell_3 = 0
counter_blob_1 = 0
counter_blob_2 = 0
counter_blob_3 = 0

for word in errors:
    if word in spell_set_1: counter_spell_1 += 1 
    if word in spell_set_2: counter_spell_2 += 1
    if word in spell_set_3: counter_spell_3 += 1
    if word in blob_set_1: counter_blob_1 += 1
    if word in blob_set_2: counter_blob_2 += 1
    if word in blob_set_3: counter_blob_3 += 1
print (counter_spell_1, counter_spell_2, counter_spell_3)
print (counter_blob_1, counter_blob_2, counter_blob_3)
    

In [ ]:
print(len(errors_raw), len(errors), len(corrections))

In [ ]:
print(len(corrections))

In [ ]:
for error, correction in zip(errors, corrections):
    if error == correction:
        display_words_in_context(test_words_3, [error], False, 20)

In [ ]:
for word in errors:
    if word not in text:
        display_words_in_context(test_words_3, [word], False, 6)

In [ ]:
display_text_in_context(text, ["never forgive me"])

In [ ]:
print([word in blob_set_3 for word in errors])

In [ ]:
where_is("bethat")

In [ ]:
"be that" in corrections

In [ ]:
"allshot" in both_3

In [ ]:
display_text_in_context(text,["ell"])

In [ ]:
replacements = {error: correction for error, correction in zip (errors, corrections)}

In [ ]:
print(replacements["with-a"])

In [ ]:
print(' '.join(word for word in (blob_only_2 - blob_only_1)))
display_words_in_context(test_words_2, blob_only_2 - blob_only_1, False, 3)

In [ ]:
print(' '.join(word for word in (spell_only_3 - spell_only_2 - spell_only_1)))
display_words_in_context(test_words_3, spell_only_3 - spell_only_2 - spell_only_1, False, 3)

In [ ]:
print(' '.join(word for word in (blob_only_3 - blob_only_2 - blob_only_1)))
display_words_in_context(test_words_3, blob_only_3 - blob_only_2 - blob_only_1, False, 3)

In [ ]:
counter = 0
for word in blob_set_2:
    if word.lower() in blob_set_3:
        print(f"{word} in both")
        print(Word(word).spellcheck())
        if word != word.lower():
            print(Word(word.lower()).spellcheck())
    else:
        print (f"{word} in 2 but not in 3")
        print(Word(word).spellcheck())
    print("\n")    
    counter = counter + 1
    if counter > 15: break    

In [ ]:
errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [ ]:
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
for word in bad_words:
    print(word in errors_split)

In [ ]:
suggestions_split = suggestions.split(' ')
suggestions_sorted = sorted(suggestions_split)
print(suggestions_sorted)

In [ ]:
errors_sorted = sorted(errors_split)
print (errors_sorted)

In [ ]:
for word in bad_words:
    errors_split.append(word)
print(errors_split)

In [ ]:
print(suggestions)

In [ ]:
errors_split = ['istrue', 'canbe', 'im', 'tj1e', 'overath', 'laval', 'itsrather', 'wer', 'silverss', 'gotto', 'thelikes', 'ifi', '0f', 'prise', 'ritz', 'ofthe', 'andthe', 'lto', 'lookingback', 'al1', 'verypleased', 'het', 'thevery', 'agre', 'bethat', 'seehe', 'ot', 'selfvident', 'sleightsofhand', 'cansee', 'thatsall', 'distinguishedand', 'villagesir', 'docrucially', 'arec', 'farradayscircle', 'witha', 'canonly', 'barnets', 'redding', 'tosit', 'clementss', 'newtmating', 'ill1terest', 'anysuch', 'dh', 'lewiss', 'donttake', 'allhis', 'contemplatin8', 'startseeing', 'symons', 'owardst', 'imvery', 'simplyaccepting', 'selftraining', 'wellcontented', 'evercourteous', 'donttake', 'allhis', 'lastminute', 'civvy', 'ohnothing', 'iarrived', 'friendsand', 'allshot']

In [ ]:


for bad_word in bad_words:
    ind = text.find(bad_word)
    if ind == -1:
        print(f"{bad_word} not found")
    else:
        window = text[ind-50:ind+50]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    #test_result = (Word(test_word).spellcheck())
    #print(test_result)


In [ ]:
test_words = text.translate(str.maketrans('', '', string.punctuation)).lower().split()
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
bad_words = ["ell", "ting"]
for bad_word in bad_words:
    try:
        ind = test_words.index(bad_word)
        print(f"The index of '{bad_word}' is: {ind}")
        print(' '.join(test_words[ind - 10:ind + 10]))
    except ValueError:
        print(f"'{item_to_find}' not found in the list.")


In [ ]:
print(' '.join(blob_only))

In [ ]:
both = spell_set & blob_set
print(len(both))

In [ ]:
print(' '.join(word for word in both))

In [ ]:
original_errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [ ]:
original_errors_split = set(original_errors.split())

In [ ]:
missed = {word for word in original_errors_split if word not in errors}

In [ ]:
print ([(word, word in blob_set_3) for word in missed])

In [ ]:
print ([(word, word in both_3) for word in missed])

In [ ]:
print ([(word, word in spell_set_3) for word in missed])


In [ ]:
for word in errors_split:
    if word not in text:
        if word not in cleaned_suggestions:
            print(word)


In [ ]:
where_is("distinguishedand")

In [ ]:
print([word for word in blob_set_3 if word not in blob_only])

In [ ]:
suggestions = "Overath Laval 1e with-a it'srather Silvers's Farraday'scircle Barnet's Clements's newt-mating Lewis's don'ttake I'mvery self-training well-contented ever-courteous don'ttake last-minute Ritz Redding self-vident sleights-of-hand be-that that'sall d-h Symons"
cleaned_suggestions = suggestions.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [ ]:
for word in suggestions.split(' '):
    if word not in text:
        print(word)

In [ ]:
for word in errors_split:
    if word not in cleaned_suggestions:`
        if word not in test_words:
            print(word)

In [ ]:
" prise " in text